# Create Respec document from ontology

This script creates a valid respec document from an ontology file.
It does so in three steps:
1. **SETUP**  
  Define target and parse ontology
1. **CREATE**  
  Create respec document adding the parsed information
1. **SAVE**  
  Save the html document into a file


## Setup 

In [ ]:
from rdflib import Graph, URIRef, BNode
from rdflib.namespace import RDF, OWL, RDFS, SKOS

import os

target = os.environ.get('VSSO_TARGET')

if not target:
    target = "vsso-core"

ns_vsso = "https://github.com/w3c/vsso#"
ns_vsso_core = "https://github.com/w3c/vsso-core#"

g = Graph()
if target == "vsso-core":
    g.parse ('../../spec/vsso-core.ttl')
else:
    g.parse ('../../spec/vss_rel_2.2.ttl')

### Parse ontology information

The following information are used from an `owl:Ontology`:

| Ontology definition  | Respec | Cardinality
|---|---|---|
| [`dct:title`](http://purl.org/dc/terms/title)  | `title`  | one |
| [`dct:abstract`](http://purl.org/dc/terms/abstract) | `abestract` | one |
| [`dct:creator`](http://purl.org/dc/terms/creator) | `editor` | multiple |
| [`dct:contributor`](http://purl.org/dc/terms/contributor) | `contributor` | multiple |
| [`dct:versionInfo`](http://purl.org/dc/terms/versionInfo) | `version`  | one |
| [`vann:preferredNamespaceUri`](http://purl.org/vocab/vann/preferredNamespaceUri) | `editor` | one |
| `@prefix` | normative namespaces | multiple |




In [ ]:
onto = None
output = {}

if (None, RDF.type, OWL.Ontology ) in g:
    print ("ontology definition found")
    onto = g.value(None,RDF.type, OWL.Ontology)
    print ("ontology: ", onto)
    
    title = g.value(onto, URIRef('http://purl.org/dc/terms/title'))
    print ("title: ", title)
    output['title'] = title

    abstract = g.value(onto, URIRef('http://purl.org/dc/terms/abstract'))
    print ("abstract: ", abstract)
    output['abstract'] = abstract.toPython()

    editors = []
    for editor in  g.objects(onto, URIRef('http://purl.org/dc/terms/creator')):
        editors.append({"name": editor.toPython()})
    print ("editors: ", editors)
    output['editors'] = editors

    contributors = []
    for contributor in  g.objects(onto, URIRef('http://purl.org/dc/terms/contributor')):
        contributors.append({'value': contributor.toPython()})
    print ("contributors: ", contributors)
    output['contributors'] = contributors

    version = g.value(onto, OWL.versionInfo)
    print ("version: ", version)
    output['version'] = version

    preferred_namespace = g.value(onto, URIRef('http://purl.org/vocab/vann/preferredNamespaceUri'))
    print ("preferred_namespace: ", preferred_namespace)
    output['preferred_namespace'] = preferred_namespace

namespaces = []
for ns_pre, ns_url in g.namespaces():
    namespaces.append ({ns_pre: ns_url})

print ("namespaces: ", namespaces)
output['namespaces'] = namespaces
    

### Classes

The following concepts are parsed and supported:

- `owl:Class`
- `rdfs:subClassOf`
- `rdfs:comment`
- `rdfs:label`
- `skos:altLabel`
- `owl:Restrictions` modelled as subclassed blank nodes with `owl:onProperty`, `owl:allValuesFrom`, `owl:someValuesFrom`
- `rdfs:seeAlso`


In [ ]:
def get_link_name_pair (uri_string):
    """
        Get link/name pair from URI string
        Limited to `#` separators
    """
    print (uri_string)
    if ns_vsso in uri_string and "vsso" == target:
        name = uri_string.split('#')[1]
        print (f"#{name}",name)
        return (f"#{name}",name)
    if ns_vsso_core in uri_string and "vsso-core" == target:
        name = uri_string.split('#')[1]
        print (f"#{name}",name)
        return (f"#{name}",name)
    return (uri_string, uri_string)
    

In [ ]:
owl_classes = {}

for concept in g.subjects(RDF.type, OWL.Class ):
    out_concept = {}
    out_concept['name'] = concept.split("#")[1]
    out_concept['uri'] = concept
    out_concept['vss_value'] = ""
    if g.value(concept, URIRef(f"{ns_vsso_core}vssFacetedClassification")):
        out_concept['vss_value'] = g.value(concept, URIRef(f"{ns_vsso_core}vssFacetedClassification")).toPython()
    out_concept['label'] = g.value(concept, RDFS.label).toPython()
    out_concept['definition'] = g.value(concept, SKOS.definition).toPython()
    concept_subClassOf = g.objects(concept, RDFS.subClassOf)
    out_concept["restrictions"] = []
    sub_classes = []
    if concept_subClassOf:
        for sc in concept_subClassOf:
            if type(sc) == BNode and OWL.Restriction == g.value(sc, RDF.type):
                restriction = {}
                print (g.value(sc, OWL.onProperty).toPython())
                restriction ['onProp'] = g.value(sc, OWL.onProperty).toPython()
                restriction ['only'] = [x.toPython() for x in g.objects(sc, OWL.allValuesFrom)]
                print (restriction ['only'])
                restriction ['oneOf'] = [x.toPython() for x in g.objects(sc, OWL.someValuesFrom)]
                print (restriction ['oneOf'])
                out_concept['restrictions'].append(restriction)
            else:
                sub_classes.append(sc.toPython())
    out_concept['subClassOf'] = sub_classes
    out_concept['seeAlso'] = [x.toPython() for x in g.objects(concept, RDFS.seeAlso)]
    owl_classes[concept] = out_concept
    

output['owl_classes'] = owl_classes



### Properties

The following concepts are parsed and supported:

- `owl:ObjectProperty`
- `owl:DatatypeProperty`
- `owl:AnnotationProperty`
- `rdfs:domain`
- `rdfs:range`
- `rdfs:comment`
- `rdfs:label`
- `skos:altLabel`
- `rdfs:seeAlso`

In [ ]:
owl_properties = {}

def prop_handling (property):
    out_prop = {}
    out_prop['name'] = property.split("#")[1]
    out_prop['uri'] = property
    out_prop['label'] = g.value(property, RDFS.label).toPython()
    out_prop['vss_value'] = ""
    if g.value(property, URIRef(f"{ns_vsso_core}vssFacetedClassification")):
        out_prop['vss_value'] = g.value(property, URIRef(f"{ns_vsso_core}vssFacetedClassification")).toPython()
        
    if g.value(property, SKOS.definition):  
        out_prop['definition'] = g.value(property, SKOS.definition).toPython()
    out_prop['seeAlso'] = [x.toPython() for x in g.objects(property, RDFS.seeAlso)]
    out_prop['domain'] = [x.toPython() for x in g.objects(property, RDFS.domain)]
    print(out_prop['domain'])
    out_prop['range'] = [x.toPython() for x in g.objects(property, RDFS.range)]
    print(out_prop['range'])
    


    return out_prop

for o_prop in g.subjects(RDF.type, OWL.ObjectProperty):
    print (o_prop)
    out_prop = prop_handling (o_prop)
    out_prop['type'] = "OWL Object Property"
    owl_properties[o_prop] = out_prop

for d_prop in g.subjects(RDF.type, OWL.DatatypeProperty):
    print (d_prop)
    out_prop = prop_handling (d_prop)
    out_prop['type'] = "OWL Datatype Property" 
    owl_properties[d_prop] = out_prop

for a_prop in g.subjects(RDF.type, OWL.AnnotationProperty):
    print (a_prop)
    out_prop = prop_handling (a_prop)
    out_prop['type'] = "OWL Annotation Property" 
    owl_properties[a_prop] = out_prop

output['owl_properties'] = owl_properties

## HTML output

### Header defintion

In [ ]:
import markdown

In [ ]:
header = f"""
<head>
    <meta charset='utf-8'>
    <script src='https://www.w3.org/Tools/respec/respec-w3c' async class='remove'></script>
    <script class='remove'>
      // All config options at https://respec.org/docs/
      var respecConfig = {{
        // Working Groups ids at https://respec.org/w3c/groups/
        group: "auto",
        specStatus: "ED",
        editors:  [{{
	  name: 'Benjamin Klotz',
	  w3cid: 91177,
	}},
	{{
	  name: 'Raphaël Troncy',
	  company: 'Institut Mines-Télécom',
	  w3cid: 38257,
	}},
	{{
	  name: 'Daniel Wilms',
	  company: 'BMW',
	  w3cid: 114129,
	}}],,
        otherLinks: [{{
          key: "Contributors",
          data: {output['contributors']}
        }}],
        github: {{
          branch: "gh-pages",
          repoURL: "w3c/vsso", 
        }},
        // See https://respec.org/docs/#xref for usage.
        // xref: "auto",
      }};
    </script>
  </head>"""


### Intro

In [ ]:
intro = f"""
<body> 
    <h1 id="title">{output['title']}</h1>
    <section id='abstract'>
      <p align="justify">
        {output['abstract']}
      </p>
    </section>
    <section id='sotd'>
      
    </section>
    """

### Namespaces

In [ ]:
namespace_table = ""
for ns in output['namespaces']:
    for ns_pre, ns_url in ns.items():
        namespace_table += f"<tr><td>{ns_pre}</td><td>{ns_url}</td><td></td></tr>\n"

namespaces_html = f"""
<section id="namespaces">

    <h2>Namespaces</h2>

    <p>The namespace for VSSo Core is <code>{output['preferred_namespace']}</code>.
        VSSo Core serves as a domain ontology and relies on the following other ontologies: 
        <section>
          <h3>Normative namespaces</h3>
          <p>Namespaces and prefixes used in normative parts of this recommendation are shown in the following table.</p>

          <table id="table-namespaces" class="simple">
            <thead><tr><th>Prefix</th><th>Namespace IRI</th><th>Source</th></tr></thead>
            <tbody>
            {namespace_table}
            </tbody>
           </table>
        </section>
    </p>
</section>
"""

### Classes

In [ ]:
owl_classes_html = "" 

for i, owl_class in output['owl_classes'].items():
    subClass_row = ""
    if "subClassOf" in owl_class.keys():
        for sc in owl_class['subClassOf']:
            sc_uri, sc_name = get_link_name_pair(sc) 
            subClass_row += f"""<tr><th class="prop">Sub-class of:</th><td><a href="{sc_uri}">{sc_name}</a></td></tr>
            """
    restriction_row = ""
    for owl_class_restriction in owl_class['restrictions']:
        print ("restrictions")
        if owl_class_restriction['only'] != [] and owl_class_restriction['onProp'] != "":
            print ("only")
            objects = ""
            for os in owl_class_restriction['only']:
                o_uri, o_name = get_link_name_pair(os)
                objects += f"""<a href="{o_uri}">{o_name}</a> """
            restriction_row += f"""<tr><th class="prop">Restrictions:</th><td>Property {owl_class_restriction['onProp']} <b>Only</b> {objects}</td></tr>
            """
            print (restriction_row)
        if owl_class_restriction['oneOf'] and owl_class_restriction['onProp']:
            print ("oneOf")
            o_uri, o_name = get_link_name_pair(owl_class_restriction['onProp'])
            restriction_row += f"""<tr><th class="prop">Restrictions:</th><td><b>Only</b> <a href="{o_uri}">{o_name}</a>/td></tr>
            """
    seeAlso_row = ""
    for owl_class_seeAlso in owl_class['seeAlso']:
        seeAlso_row += f"""<tr><th class="prop">See also:</th><td><a href="{owl_class_seeAlso}">{owl_class_seeAlso}</a></td></tr>"""
    vss_row = ""
    if owl_class['vss_value']:
        vss_row = f"""<tr><th class="prop">VSS path:</th><td>{owl_class['vss_value']}</td></tr>
        """

              
    owl_classes_html += f"""
    <section id="{owl_class['name']}">
        <h4>{owl_class['name']}</h4>
        <table class="def">
            <tr><th>Owl Class:</th><td><a href="#{owl_class['name']}"><code>vsso:{owl_class['name']}</code></a></td></tr>
            <tr><th class="prop">Definition:</th><td>{markdown.markdown(owl_class['definition'])}</td></tr>
            {subClass_row}
            {vss_row}
            {seeAlso_row}
            {restriction_row}
        </table>
    </section>
    """



### Properties

In [ ]:
owl_properties = ""

if output['owl_properties']:
    owl_properties += """
        <section id="vocabulary-specification-properties">
            <h3>OWL Properties</h3>"""

    for i, owl_prop in output['owl_properties'].items():

        prop_uri, prop_name = get_link_name_pair(owl_prop['uri'])

        seeAlso_row = ""
        for owl_prop_seeAlso in owl_prop['seeAlso']:
            seeAlso_row += f"""<tr><th class="prop">See also:</th><td><a href="{owl_prop_seeAlso}">{owl_prop_seeAlso}</td></tr>"""
        domain_row = ""
        for owl_prop_domain in owl_prop['domain']:
            prop_domain_uri, prop_domain_name = get_link_name_pair(owl_prop_domain) 
            domain_row += f"""<tr><th class="prop">Domain:</th><td><a href="{prop_domain_uri}">{prop_domain_name}</td></tr>"""
        range_row = ""
        for owl_prop_range in owl_prop['range']:
            prop_range_uri, prop_range_name = get_link_name_pair(owl_prop_range)
            range_row += f"""<tr><th class="prop">Range:</th><td><a href="{prop_range_uri}">{prop_range_name}</td></tr>"""
        comment_row = ""
        if "definition" in owl_prop.keys():
            comment_row = f"""<tr><th class="prop">Definition:</th><td>{markdown.markdown(owl_prop['definition'])}</td></tr>"""
        owl_properties += f"""
        <section id="{prop_name}">
            <h4>{prop_name}</h4>
            <table class="def">
                <tr><th>{owl_prop['type']}:</th><td><a href="{prop_uri}"><code>vsso:{prop_name}</code></a></td></tr>
                {comment_row}
                {domain_row}
                {range_row}
                {seeAlso_row}
            </table>
        </section>
        """
    owl_properties += "</section>"

In [ ]:
figure_html = ""
if "vsso-core" == target:
    figure_html = f"""
        <figure id="figure-vsso-uc">
            <img src="static/vsso-core.ttl.svg" alt="VSSo Core Structure" />
            <figcaption>Structure and components of VSSo Core</figcaption>
        </figure> 
    """
docs = f"""
<!DOCTYPE html>
<html>
{header}
<body>
{intro}
{namespaces_html}
<section id="vocabulary-specification">
    
    <h2>Vocabulary specification</h2>
    {figure_html}
    <section id="vocabulary-specification-classes">
        <h3>OWL Classes</h3>        
        {owl_classes_html}
    </section>
        {owl_properties}
</section>
</body> 
"""    

## SAVE

In [ ]:
if "vsso-core" == target:
    with open('../../spec/vsso-core-re.html', 'w') as f:
        f.write(docs)   
else: 
    with open('../../spec/vsso-re.html', 'w') as f:
        f.write(docs)   